In [5]:
#排網頁用的
import requests
from pyquery import PyQuery as pq
#捲網頁用的
from selenium import webdriver
from bs4 import BeautifulSoup
from time import sleep

In [6]:
res = requests.get("https://tw.buy.yahoo.com/sitemap/category-list")
doc = pq(res.text)
#用request 跟 pyquery 把資料讀進來

In [3]:
#看是不是讀到想要的資料
doc('.level3Title').text()

'智慧型手機 一般型手機 福利機 耳機 藍牙耳機 藍牙接收器 耳機擴大機 耳機週邊 MP3播放器 AirPods 錄音筆/機 智慧型手錶/手環 智慧穿戴配件 VR頭戴裝置 Apple Watch 數位相機 單眼/微單眼/類單眼 相機鏡頭 拍立得 拍立得底片 運動攝影機 數位攝影機 攝影機配件 微型攝影機 空拍機/無人機 空拍機配件 穩定器 望遠鏡 顯微鏡 放大鏡 保護殼/皮套 保護貼/螢幕貼 自拍棒 手機支架 外接鏡頭 數位週邊 防水袋 手機頸掛繩 上網卡 捲線器 觸控筆 lightning線 Micro USB線 typc-C線 HDMI/轉接頭 USB充電器 無線充電版 防潮箱 相機包 相機背帶 腳架/雲台 閃燈/手把 濾鏡/偏光鏡/UV鏡 鏡頭配件 相機週邊 相機清潔/保護貼 攝影棚器材 行動電源 充電電池 一般電池 手機電池 相機電池 充電器 變壓器 網路攝影機 寵物攝影機 智慧電子鎖 智能家居設備 空氣品質偵測器 智慧防丟定位器 智慧機器人 智能音箱 視訊會議系統 實物攝影機 居家安全 滅火器/火災警報器 防盜警報器 無線監視系統 監控攝影主機 監控攝影組合 監視器攝影機 監控居家週邊 電視 電視配件 壁掛架 電視盒/電視棒 DVD/藍光播放器 影音線材 藍牙喇叭/音響 揚聲器喇叭 音響/家庭劇院 擴大機 麥克風 卡拉OK/KTV CD播放/收音機 黑膠唱盤 投影機 布幕 投影機週邊 果汁/冰沙機 咖啡機/咖啡壺 電子鍋/電鍋 烤箱 微波爐 電磁爐 快煮壺/熱水瓶 開飲機 氣泡水機 洗碗/烘碗機 淨水設備/配件 瓦斯爐 除油煙機 烤/製麵包機 氣炸鍋/油炸鍋 點心家電 廚房小家電/配件 電話/對講機 熨斗 掛燙機 捕蚊燈 驅蟲/驅鼠器 縫紉機 檯燈 保險箱/櫃 電動工具 超音波清洗機 熱水器 免治馬桶 生活家電配件 雙11福袋組 吸塵器 掃地機器人 除蟎機 蒸氣拖把 吸塵器耗材/配件 掃地機器人耗材 除濕機 空氣清淨機 烘衣/烘鞋/烘被機 吹風機 電捲棒 電動牙刷 電鬍刀 美體刀/除毛刀/冰刀 電動鼻毛刀 洗臉機/美容儀 剪髮器 沙發按摩椅 按摩家電 肩頸腰背按摩 足部美腿機 桑拿機/三溫暖 泡腳機/足浴機 冷氣 電風扇 電暖器 電熱毯 隨身暖蛋/懷爐 變頻300L以下 變頻301-599L 變頻600L以上 定頻300L以下 定頻301-599L 定頻6

In [20]:
dataList = []
doc.make_links_absolute(base_url=res.url)
driver = webdriver.Chrome("D:\jupyter notebook\程式設計概論\chromedriver.exe")
for eachMainDoc in doc(".level3Title").items():
    print(eachMainDoc.attr("href"))
    mainRes = requests.get(eachMainDoc.attr("href"))
    mainDoc = pq(mainRes.text)
    mainDoc.make_links_absolute(base_url=mainRes.url)
    #選出每個類型的分頁連結
    nextDoc = mainDoc
    nextRes = mainRes
    driver.get(eachMainDoc.attr("href"))
    while True:
        nextDoc.make_links_absolute(base_url=nextRes.url)
        wrapper = driver.find_element_by_css_selector("#isoredux-root")
        lastHeight = -1
        for i in range(10):
            height = wrapper.size['height']
            if lastHeight == height:
                continue
            lastHeight = height
            driver.execute_script('window.scrollTo(0, {})'.format(height))
            sleep(0.5)
        for eachItemDoc in nextDoc(".BaseGridItem__itemInfo___3E5Bx").items():
            itemDict = {}
            itemDict["title"] = eachItemDoc(".BaseGridItem__title___2HWui").text()
            itemDict["price"] = float(eachItemDoc(".BaseGridItem__price___31jkj").text().replace("$","").replace(",",""))
            dataList.append(itemDict)
        print(nextDoc(".Pagination__wrap___FpaCH.Pagination__hideOnMobile___3bl9T>.Pagination__numberBtn___3HrVf").attr("href"))
        webUrl = nextDoc(".Pagination__wrap___FpaCH.Pagination__hideOnMobile___3bl9T>.Pagination__numberBtn___3HrVf").attr("href")            
        if not webUrl:
            break
        nextRes = requests.get(webUrl)
        nextDoc = pq(nextRes.text)

https://tw.buy.yahoo.com/category/4385945
None
https://tw.buy.yahoo.com/category/4385944
None
https://tw.buy.yahoo.com/category/31369132
None
https://tw.buy.yahoo.com/category/4387250


WebDriverException: Message: chrome not reachable
  (Session info: chrome=79.0.3945.88)


In [8]:
print(dataList)

[{'title': 'Apple iPhone 11 128G 6.1吋智慧型手機', 'price': 26488.0}, {'title': '紅米 Redmi Note 8T (4G/64G) 6.3吋八核心智慧手機', 'price': 5299.0}, {'title': '紅米 Redmi Note 8 Pro (6G/64G) 6.53吋八核心手機', 'price': 6599.0}, {'title': 'Apple iPhone 11 Pro Max 256G 6.5吋智慧型手機', 'price': 41999.0}, {'title': 'SAMSUNG Galaxy A70 6G/128G 6.7吋大廣角手機', 'price': 980010280.0}, {'title': 'Apple iPhone 11 128G 6.1吋智慧型手機', 'price': 2658826900.0}, {'title': 'NOKIA 2720 (512MB/4G) 經典折疊式手機 超長續航', 'price': 2990.0}, {'title': 'NOKIA 2720 (512MB/4G) 經典折疊式手機 超長續航-紅色', 'price': 2990.0}, {'title': 'iNO CP300 雙螢幕銀髮族御用4G摺疊手機(公司貨)', 'price': 1840.0}, {'title': 'Panasonic VS-200雙大畫面2.8吋4G御守機 老人機 VS200', 'price': 2390.0}, {'title': 'GPLUS GH7200 雙卡雙待摺疊式科園機(部隊版)', 'price': 880.0}, {'title': 'Panasnoic VS-200 4G大字幕大聲音折疊機', 'price': 2690.0}, {'title': '【福利品】Apple iPhone 8 Plus 64GB', 'price': 1261613280.0}, {'title': '【福利品】Apple iPhone X 64G 5.8吋智慧型手機', 'price': 1425015000.0}, {'title': '【福利品】Apple iPhone 8 64GB 智慧手機', 'price': 99751050

In [3]:
driver = webdriver.Chrome("D:\jupyter notebook\程式設計概論\chromedriver.exe")
driver.get('https://tw.buy.yahoo.com/category/4385945')
wrapper = driver.find_element_by_css_selector("#isoredux-root")

In [4]:
lastHeight = -1
for i in range(10):
    height = wrapper.size['height']
    if lastHeight == height:
        break
    lastHeight = height
    driver.execute_script('window.scrollTo(0, {})'.format(height))
    sleep(0.5)
    print(height)

7224
7223
7213
7206


In [ ]:
dataList = []
doc.make_links_absolute(base_url=res.url)
for eachMainDoc in doc("#side-menu > li:nth-child(n+2) > a").items():
    #print(eachMainDoc.attr("href"))
    mainRes = requests.get(eachMainDoc.attr("href"))
    mainDoc = pq(mainRes.text)
    mainDoc.make_links_absolute(base_url=mainRes.url)
    for eachSubDoc in mainDoc("#side-menu > li.active > ul > li > a").items():
        print(eachSubDoc.attr("href"))
        subRes = requests.get(eachSubDoc.attr("href"))
        subDoc = pq(subRes.text)
        #part 2
        print(len(subDoc(".BaseGridItem__content___3LORP BaseGridItem__hover___3UlCS")))
        nextDoc = subDoc
        nextRes = subRes
        while True:
            nextDoc.make_links_absolute(base_url=nextRes.url)
            for eachItemDoc in nextDoc(".test-site .thumbnail").items():
                itemDict = {}
                itemDict["title"] = eachItemDoc(".caption .title").text()
                itemDict["price"] = float(eachItemDoc(".caption > .price").text().replace("$",""))
                itemDict["stars"] = int(eachItemDoc(".ratings>p:nth-child(2)").attr("data-rating"))
                dataList.append(itemDict)
            print(nextDoc(".pagination>li.active+li>a").attr("href"))
            webUrl = nextDoc(".pagination>li.active+li>a").attr("href")
            if not webUrl:
                break
            nextRes = requests.get(webUrl)
            nextDoc = pq(nextRes.text)